In [293]:
import pandas as pd
import datetime
import numpy as np
import glob
import plotly.express as px
import re

%matplotlib inline

**1. Загрузка данных в df из .csv**

In [283]:
month = datetime.datetime.today().strftime('%Y.%m')
day = '2023.04.19'  # datetime.datetime.today().strftime('%Y.%m.%d')

# заменить при необходимости
files_folder = f'/Users/dmitrijdolgopolov/Documents/1_study_python/projects/\
1_avito_parsing/flats_from_avito/tables_by_districts_{month}/{day}/All_flats'

files = glob.glob(f'{files_folder}/*.csv')
file_name = files[0]

In [284]:
df = pd.read_csv(file_name).drop(columns='Unnamed: 0')

**2. Проверка данных в df**

In [323]:
df.head()

,url,Cost,Rooms,Square,Cost_per_m2,Floor,Floors_in_house,District,Address,Metro,Metro_distance,Date,Date_note
0,https://www.avito.ru/sankt-peterburg/kvartiry/...,9700000,3,71.8,135097,3,9,Невский,"Искровский пр-т, 20",Улица Дыбенко,11–15 мин.,19.04.2023,2 часа назад
1,https://www.avito.ru/sankt-peterburg/kvartiry/...,3885000,Студия,24.2,160537,3,3,Невский,"пр-т Обуховской Обороны, 123А",Елизаровская,16–20 мин.,19.04.2023,6 часов назад
2,https://www.avito.ru/sankt-peterburg/kvartiry/...,4600000,Студия,27.7,166064,15,15,Невский,"ул. Крыленко, 14с2",Улица Дыбенко,21–30 мин.,19.04.2023,1 час назад
3,https://www.avito.ru/sankt-peterburg/kvartiry/...,7500000,1,34.5,217391,11,25,Невский,"ул. Коллонтай, 5/1",Проспект Большевиков,16–20 мин.,18.04.2023,1 день назад
4,https://www.avito.ru/sankt-peterburg/kvartiry/...,10100000,1,47.5,212631,16,28,Невский,"Рыбацкий пр-т, 18к2",Рыбацкое,16–20 мин.,19.04.2023,18 часов назад


In [287]:
df.duplicated().sum()

0

In [289]:
df.isna().sum()

url                0
Cost               0
Rooms              0
Square             0
Cost_per_m2        0
Floor              0
Floors_in_house    0
District           0
Address            0
Metro              0
Metro_distance     0
Date               0
Date_note          0
dtype: int64

In [290]:
df.dtypes

url                 object
Cost                 int64
Rooms               object
Square             float64
Cost_per_m2          int64
Floor                int64
Floors_in_house      int64
District            object
Address             object
Metro               object
Metro_distance      object
Date                object
Date_note           object
dtype: object

In [353]:
df.Rooms.unique()

array(['3', 'Студия', '1', '2', '4', '5', 'Доля в кв.',
       'Своб. планировка', '7', '10 и более', '6', '8', '> 9', '9', 3,
       'Комната в кв.', 1], dtype=object)

In [352]:
df.Floor.unique()

array([ 3, 15, 11, 16, 14,  6,  4,  5,  9, 10,  2,  1,  7,  8, 12, 19, 13,
       22, 17, 23, 24, 21, 18, 20, 31, 26, 28, 32, 25, 29, 34, 27, 30])

In [329]:
df.Floors_in_house.unique()

array([ 9,  3, 15, 25, 28, 16, 20,  5, 23, 24, 17, 10, 19, 12, 18, 14, 26,
        6, 13, 22,  4,  8,  2, 32, 27, 36, 29, 11,  7, 21, 35, 34, 37, 33,
        1, 30])

In [306]:
# df.loc[(df['Rooms'].str.contains(r'Аукцион: \d')) | (df['Rooms'] == 'Квартира')].to_csv('fix.csv')

In [350]:
# df.loc[df.Rooms == 'Квартира']#.to_csv('fix.csv')

,url,Cost,Rooms,Square,Cost_per_m2,Floor,Floors_in_house,District,Address,Metro,Metro_distance,Date,Date_note


In [354]:
# исправил неправильную выгрузку при парсинге
# df.loc[14972, 'Rooms'] = 1

In [319]:
def rooms_bug_find(x):
    'Исправляет количество комнат, удаляя префикс "Аукцион: "'
    pattern = re.compile(r'Аукцион: \d')
    result = pattern.search(x)
    if result is not None:
        new_x = result[0].split()[-1]
        return new_x
    else:
        return x

In [324]:
df.Rooms = df.Rooms.apply(rooms_bug_find)

In [325]:
def fix_floor(x):
    'Исправляет опечатку в этажности'
    if x == 99 or x == '99':
        x = 9
        return x
    else:
        return x

In [326]:
# исправляет опечатку в этажности
df.Floors_in_house = df.Floors_in_house.apply(fix_floor)

In [327]:
# сбор индексов
df = df.reset_index(drop=True)

In [355]:
# floor, floors_in_house в int
df[['Floor', 'Floors_in_house']] = df[['Floor', 'Floors_in_house']].astype('int')

In [356]:
df.head()

,url,Cost,Rooms,Square,Cost_per_m2,Floor,Floors_in_house,District,Address,Metro,Metro_distance,Date,Date_note
0,https://www.avito.ru/sankt-peterburg/kvartiry/...,9700000,3,71.8,135097,3,9,Невский,"Искровский пр-т, 20",Улица Дыбенко,11–15 мин.,19.04.2023,2 часа назад
1,https://www.avito.ru/sankt-peterburg/kvartiry/...,3885000,Студия,24.2,160537,3,3,Невский,"пр-т Обуховской Обороны, 123А",Елизаровская,16–20 мин.,19.04.2023,6 часов назад
2,https://www.avito.ru/sankt-peterburg/kvartiry/...,4600000,Студия,27.7,166064,15,15,Невский,"ул. Крыленко, 14с2",Улица Дыбенко,21–30 мин.,19.04.2023,1 час назад
3,https://www.avito.ru/sankt-peterburg/kvartiry/...,7500000,1,34.5,217391,11,25,Невский,"ул. Коллонтай, 5/1",Проспект Большевиков,16–20 мин.,18.04.2023,1 день назад
4,https://www.avito.ru/sankt-peterburg/kvartiry/...,10100000,1,47.5,212631,16,28,Невский,"Рыбацкий пр-т, 18к2",Рыбацкое,16–20 мин.,19.04.2023,18 часов назад


In [367]:
df.Date.unique()

array(['19.04.2023', '18.04.2023', '17.04.2023', ..., '21 марта 00:07',
       '21 марта 02:08', '21 марта 09:22'], dtype=object)

In [365]:
def convert_month(date):
    date_pattern = r'(\d{1,2})\s(\w+)\s\d{1,2}:\d{1,2}'
    re_result = re.findall(date_pattern, date)
    if re_result is not None:
        print(re_result)
        item_day = re_result[0][0]
        item_month = convert_month(re_result[0][1])
        year = datetime.date.today().year

        dict_of_months = {'января': '01', 'февраля': '02', 'марта': '03', 'апреля': '04', 'мая': '05', 'июня': '06',
                          'июля': '07', 'августа': '08', 'сентября': '09', 'октября': '10', 'ноября': '11', 'декабря': '12'}
        for el in dict_of_months.keys():
            if month_name in el:
                return dict_of_months[el]
    else:
        return date

In [366]:
df.Date.apply(convert_month)

[]


IndexError: list index out of range

In [357]:
'Повторное сохранение исправленного датафрейма в csv файл'
df.to_csv(file_name)

**Девятиэтажные дома**
как Подвойского, д. 17, к. 1

In [181]:
# фильтрую по 9-этажным домам
df_9_floors_houses = df.query('Floors_in_house == 9')

In [184]:
# число объявлений
df_9_floors_houses.url.nunique()

3965

In [208]:
# характеристики цен за 1 м2
mean = df_9_floors_houses.query('District == "Невский"').Cost_per_m2.mean()
median = df_9_floors_houses.query('District == "Невский"').Cost_per_m2.median()
q_25 = df_9_floors_houses.query('District == "Невский"').Cost_per_m2.quantile(0.25)
q_50 = df_9_floors_houses.query('District == "Невский"').Cost_per_m2.quantile(0.5)
q_7 = int(df_9_floors_houses.query('District == "Невский"').Cost_per_m2.quantile(0.7))

In [236]:
my_df = df_9_floors_houses.loc[((df_9_floors_houses['Cost_per_m2'] > q_7) & 
                        (df_9_floors_houses['Cost_per_m2'] < 200000) &
                       (df_9_floors_houses['Rooms'] == '2'))]\
                  .query('District == "Невский"')\
                  .sort_values('Cost_per_m2')

In [233]:
my_df.to_csv('Конкуренты_Подвойского.csv')

In [237]:
175000 * 44.1

7717500.0

In [134]:
df_9_floors_houses.groupby('District', as_index=False)\
                  .agg({'Cost_per_m2': [lambda x: x.quantile(0.25),
                                       lambda x: x.quantile(0.5),
                                       lambda x: x.quantile(0.75)]})\
                  .rename(columns={'<lambda_0>': 'q_0.25',
                                  '<lambda_1>': 'q_0.5',
                                  '<lambda_2>': 'q_0.75'})

District Cost_per_m2                     
                           q_0.25     q_0.5     q_0.75
0      Адмиралтейский   300195.75  370302.0  414541.25
1    Василеостровский   192120.00  226544.0  272294.00
2          Выборгский   145387.75  164494.5  199006.50
3         Калининский   147632.75  161882.0  178571.00
4           Кировский   143405.00  158677.0  182611.75
5   Красногвардейский   143914.50  158129.5  177803.00
6      Красносельский   131121.00  144776.0  161914.00
7          Московский   173367.50  201508.5  237568.50
8             Невский   142963.00  155183.5  173569.25
9       Петроградский   340149.75  406898.0  577774.75
10         Приморский   158333.00  175356.5  195878.00
11        Фрунзенский   146687.75  161069.5  179922.75
12        Центральный   323827.75  380900.0  508410.25

In [74]:
7600000 / 44.1

172335.60090702947

In [91]:
173000 * 44.1

7629300.0

In [114]:
# нахожу квантили
quantiles = df_9_floors_houses.Cost_per_m2.quantile([0.25, 0.5, 0.75, 0.815])
quantiles

0.250    145928.50
0.500    155577.00
0.750    166312.00
0.815    172889.64
Name: Cost_per_m2, dtype: float64

In [115]:
df_9_floors_houses[df_9_floors_houses.Cost_per_m2 > quantiles[0.750]] \
    .sort_values('Cost_per_m2', ascending=False) \
    .to_csv('''/Users/dmitrijdolgopolov/Documents/1_study_python/projects/1_avito_parsing/flats_from_avito/top/9_floors_houses_0.75_percentile.csv''')

In [68]:
df_9_floors_houses.head()

,url,Cost,Rooms,Square,Floor,Floors_in_house,Address,Metro,Date,Date_note,cost_per_m2
1,https://www.avito.ru/sankt-peterburg/kvartiry/...,7279000,2,50.6,8,9,"ул. Дыбенко, 12к3",Улица Дыбенко,13.04.2023,39 минут назад,143853
6,https://www.avito.ru/sankt-peterburg/kvartiry/...,4562790,2,50.2,6,9,"Товарищеский пр-т, 2к2",Проспект Большевиков,13.04.2023,35 минут назад,90892
9,https://www.avito.ru/sankt-peterburg/kvartiry/...,6800000,2,50.9,5,9,"ул. Дыбенко, 12к1",Улица Дыбенко,13.04.2023,12 часов назад,133595
15,https://www.avito.ru/sankt-peterburg/kvartiry/...,8000000,2,50.2,5,9,"Санкт-Петербург, Товарищеский проспект, 2к2",Проспект Большевиков,12.04.2023,1 день назад,159362
31,https://www.avito.ru/sankt-peterburg/kvartiry/...,7350000,2,51.0,3,9,"ул. Подвойского, 17к2",Улица Дыбенко,07.04.2023,6 дней назад,144117


7688279.376712329